# Setup

# ☀️ Solar Panel & Boiler Detection in Madagascar - Satellite & Drone Imagery Challenge

## 📜 Competition Overview

Access to reliable and sustainable energy is a critical issue in Madagascar and across Africa, where many communities either lack electricity or rely on costly, environmentally harmful energy sources. Solar technology offers a promising alternative — but identifying and tracking the adoption of solar panels and boilers over large, remote areas remains a major challenge.

The goal of this competition is to develop a **machine learning model** capable of **detecting and counting** solar panels and solar boilers in satellite and drone imagery of Madagascar.

A robust detection system has important real-world applications, including:
- Supporting governments and NGOs with energy planning and policy
- Monitoring the impact of renewable energy programs
- Optimizing resource allocation for electrification efforts

In [ ]:
!nvidia-smi
#papapap

Fri Feb 21 20:32:20 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   50C    P8             10W /   70W |       1MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## 📦 Library Imports

We begin by importing all necessary libraries for the project, including:

- **PyTorch** for deep learning model development
- **Albumentations** for powerful image augmentations
- **TIMM** for easy access to pretrained image models
- **Pandas** and **NumPy** for data handling
- **OpenCV** and **PIL** for image processing
- **Scikit-learn** for cross-validation strategies
- **TQDM** for progress visualization during training

These tools form the foundation of the modeling pipeline for solar panel and boiler detection.


In [2]:
import os
import timm
import torch
import torch.nn as nn
import torch.optim as optim
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import cv2
from sklearn.model_selection import StratifiedKFold
import numpy as np
from tqdm import tqdm
import random
from torchvision import transforms
from PIL import Image, ImageEnhance

/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 2.0.4 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


# Imports

## 📂 Load Training Data

We load the provided training dataset, which contains:
- Image identifiers
- Corresponding labels (number of solar panels and boilers)

This structured data will be used to train and validate our model.


In [ ]:
# Read in the training dataset
train = pd.read_csv("/kaggle/input/lacuna-solar-survey-challenge/Train.csv")

train

## 🛠️ Preprocessing: Dataset Structuring

We perform preprocessing steps to better structure the dataset for model training:

- Create mappers for `placement` and `img_origin` information linked to each image ID
- Aggregate the `boil_nbr` and `pan_nbr` counts per unique image
- Merge back additional metadata (`placement` and `img_origin`)
- Generate the full file path for each image

This prepares a clean and enriched dataframe for further modeling.

In [5]:
# Create a placement mapper
placement_mapper = train[["ID", "placement"]].drop_duplicates().set_index("ID").to_dict()
# Create a "img_origin" mapper
img_origin_mapper = train[["ID", "img_origin"]].drop_duplicates().set_index("ID").to_dict()

# Group by "ID" and sum up boil_nb, pan_nbr
train_df = train.groupby("ID").sum().reset_index()[["ID", "boil_nbr", "pan_nbr"]]

# Map img_origin and placement
train_df["img_origin"] = train_df["ID"].map(img_origin_mapper["img_origin"])
train_df["placement"] = train_df["ID"].map(placement_mapper["placement"])
train_df
# Create path column
train_df["path"] = "/kaggle/input/lacuna-solar-survey-challenge/images/" + train_df["ID"] + ".jpg"
train_df[train_df['boil_nbr']!=0] 

,ID,boil_nbr,pan_nbr,img_origin,placement,path
19,ID0I1tHW,1,0,S,S-unknown,/kaggle/input/lacuna-solar-survey-challenge/im...
30,ID0POc2HN,1,0,S,S-unknown,/kaggle/input/lacuna-solar-survey-challenge/im...
32,ID0RNQgs4Y1a7,1,0,S,S-unknown,/kaggle/input/lacuna-solar-survey-challenge/im...
39,ID0YWEiDPjmAfA50E,1,0,S,S-unknown,/kaggle/input/lacuna-solar-survey-challenge/im...
44,ID0g0Yd,1,0,S,S-unknown,/kaggle/input/lacuna-solar-survey-challenge/im...
...,...,...,...,...,...,...
3287,IDzZw0Ot8e05Xu,1,0,S,S-unknown,/kaggle/input/lacuna-solar-survey-challenge/im...
3288,IDzaUuik1u5aOQt9,2,0,S,S-unknown,/kaggle/input/lacuna-solar-survey-challenge/im...
3295,IDzj0TllxhxN,1,0,S,S-unknown,/kaggle/input/lacuna-solar-survey-challenge/im...
3303,IDzpcgPa,1,0,S,S-unknown,/kaggle/input/lacuna-solar-survey-challenge/im...


In [6]:
train_df.head(2)

,ID,boil_nbr,pan_nbr,img_origin,placement,path
0,ID00rw8,0,2,D,roof,/kaggle/input/lacuna-solar-survey-challenge/im...
1,ID014O6EC7,0,1,D,roof,/kaggle/input/lacuna-solar-survey-challenge/im...


## 🧩 Cross-Validation Strategy: Stratified K-Fold

To ensure balanced distribution of multi-label targets across folds, we implement **Stratified K-Fold** cross-validation:

- Create a `stratify_label` by summing `boil_nbr` and `pan_nbr` for each sample
- Use **7 folds** with shuffling and a fixed random seed for reproducibility
- Assign a `fold` index to each sample for organized training and validation splits

This method helps maintain class balance during training and evaluation.

In [7]:
# Stratified KFold based on multi-label targets
train_df["stratify_label"] = train_df[["boil_nbr", "pan_nbr"]].sum(axis=1)
skf = StratifiedKFold(n_splits=7, shuffle=True, random_state=42)
train_df["fold"] = -1
for fold, (_, valid_idx) in enumerate(skf.split(train_df, train_df["stratify_label"])):
    train_df.loc[valid_idx, "fold"] = fold

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=7.
  warnings.warn(


## 🎨 Data Augmentation Strategies

To improve the model’s robustness and prevent overfitting, we apply a combination of **Albumentations** and **PyTorch** data augmentations:

### Custom Transformations:
- **RandomSharpen**: Randomly increases image sharpness
- **RandomBlur**: Randomly applies Gaussian blur

### Albumentations Transforms (for training and testing):
- Resizing, flipping, brightness/contrast adjustment
- Shift, scale, rotate, elastic, and grid distortions
- Normalization and conversion to tensor

### PyTorch Transforms (for training and validation):
- Resize to InceptionV3’s input size (299×299)
- Random horizontal flip, sharpening, blurring
- Normalization following ImageNet statistics

These augmentations simulate real-world conditions and improve generalization.

In [ ]:
class RandomSharpen:
    def __init__(self, probability=0.5):
        self.probability = probability

    def __call__(self, img):
        if random.random() < self.probability:
            enhancer = ImageEnhance.Sharpness(img)
            img = enhancer.enhance(random.uniform(1.5, 2.0))  # Randomly increase sharpness
        return img

class RandomBlur:
    def __init__(self, probability=0.5):
        self.probability = probability

    def __call__(self, img):
        if random.random() < self.probability:
            img = np.array(img)
            ksize = random.choice([3, 5])  # Randomly choose kernel size
            img = cv2.GaussianBlur(img, (ksize, ksize), 0)
            img = Image.fromarray(img)
        return img

# Define Albumentations transformations
albu_train_transforms = A.Compose([
    A.Resize(256, 256),  # Resize the image to 256x256
    A.HorizontalFlip(p=0.5),  # Apply horizontal flip with 50% probability
    A.RandomBrightnessContrast(p=0.2),  # Randomly adjust brightness and contrast with 20% probability
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.2, rotate_limit=15, p=0.5),  
    # Randomly shift, scale, and rotate the image
    A.GridDistortion(p=0.2),  # Apply grid distortion with 20% probability
    A.ElasticTransform(p=0.2),  # Apply elastic transform with 20% probability
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize the image
    ToTensorV2()  # Convert the image to a PyTorch tensor
])

albu_test_transforms = A.Compose([
    A.Resize(256, 256),  # Resize the image to 256x256
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize the image
    ToTensorV2()  # Convert the image to a PyTorch tensor
])

# Define PyTorch transformations
torch_train_transforms = transforms.Compose([
    transforms.Resize((299, 299)),  
    transforms.RandomHorizontalFlip(),
    RandomSharpen(probability=0.5),
    RandomBlur(probability=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

torch_valid_transforms = transforms.Compose([
    transforms.Resize((299, 299)),  # InceptionV3 expects 299x299 input size
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

## 🔀 Combined Albumentations and PyTorch Transformations

We define a custom **CombinedTransform** class to seamlessly merge **Albumentations** and **PyTorch** transformations in a single pipeline:

- First, apply powerful augmentations from Albumentations (e.g., resizing, distortions).
- Then, convert the augmented image back to a **PIL** format.
- Finally, apply additional PyTorch transformations (e.g., random blur, sharpening, normalization).

### Purpose:
This approach leverages the strengths of both libraries, ensuring rich, diverse augmentations while maintaining compatibility with PyTorch DataLoaders.


In [ ]:
# Custom transformation class to combine Albumentations and PyTorch transformations
class CombinedTransform:
    def __init__(self, albu_transform, torch_transform):
        self.albu_transform = albu_transform
        self.torch_transform = torch_transform

    def __call__(self, image):
        # Apply Albumentations transformations
        image = np.array(image)
        augmented = self.albu_transform(image=image)
        image = augmented['image']
        
        # Convert back to PIL image for PyTorch transformations
        image = transforms.ToPILImage()(image)
        
        # Apply PyTorch transformations
        image = self.torch_transform(image)
        
        return image

# Combine transformations
train_transforms = CombinedTransform(albu_train_transforms, torch_train_transforms)
valid_transforms = CombinedTransform(albu_test_transforms, torch_valid_transforms)

## 🏗️ Dataset, Early Stopping, and Model Architecture

This section builds the essential backbone for model training:

### 📦 Custom Dataset Class: `SolarPanelDataset`
- Loads images based on paths in the dataframe.
- Applies the combined Albumentations and PyTorch transformations.
- Returns either:
  - Image and target (boiler and panel counts) for training.
  - Image only for inference.

### ⏳ Early Stopping Strategy
- A custom **EarlyStopping** class monitors validation loss.
- Stops training early if no improvement is seen after a defined patience.
- Saves the best model checkpoint automatically.

### 🧠 Model Architecture: `InceptionRegressor`
- Built upon a **pretrained InceptionV3** model using **timm**.
- Modifies the final fully connected layer to output two values:
  - **Boiler count** and **Solar panel count**.

This design ensures an efficient and stable training process focused on both predictive performance and overfitting prevention.

In [ ]:
# Custom Dataset
class SolarPanelDataset(Dataset):
    def __init__(self, dataframe, transform=None, to_train=True):
        self.dataframe = dataframe
        self.transform = transform
        self.to_train = to_train

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        image = Image.open(row["path"]).convert("RGB")
        
        if self.transform:
            image = self.transform(image)

        if self.to_train:
            target = torch.tensor([row["boil_nbr"], row["pan_nbr"]], dtype=torch.float32)
            return image, target
        else:
            return image



class EarlyStopping:
    def __init__(self, patience=5, verbose=False):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False

    def __call__(self, val_loss, model, path):
        score = -val_loss
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model, path)
        elif score < self.best_score:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model, path)
            self.counter = 0

    def save_checkpoint(self, val_loss, model, path):
        if self.verbose:
            print(f'Validation loss decreased ({self.best_score:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), path)

import timm
import torch.optim as optim

class InceptionRegressor(nn.Module):
    def __init__(self):
        super(InceptionRegressor, self).__init__()
        self.model = timm.create_model("inception_v3", pretrained=True)
        # Replace the last fully connected layer
        self.model.fc = nn.Linear(self.model.fc.in_features, 2)

    def forward(self, x):
        return self.model(x)

## ⚙️ Model Training Setup and Data Preparation

This section sets up the environment for training the solar panel and boiler detection model using an InceptionV3 backbone.

### 🛠 Model and Training Configuration
- **Model**: `InceptionRegressor` based on a pretrained `inception_v3` model from `timm`.
- **Loss Function**: L1 Loss (`nn.L1Loss`) for robust regression performance.
- **Optimizer**: Adam optimizer with an initial learning rate of `1e-4`.
- **Learning Rate Scheduler**: Reduces the learning rate by a factor of 0.1 if the validation loss plateaus for 5 epochs.
- **Early Stopping**: Stops training if validation loss does not improve for 10 consecutive epochs.
- **Device**: Automatically selects GPU if available, otherwise CPU.
- **Logging**: TensorBoard `SummaryWriter` is initialized for monitoring training progress.

### 📦 Dataloader Preparation
- **Stratified K-Fold** is used for creating training and validation splits based on the sum of boiler and panel counts.
- **Fold Selection**: Fold 0 is used for validation; the remaining folds are used for training.
- **Datasets**: 
  - `SolarPanelDataset` instances apply a combination of Albumentations and TorchVision transformations.
- **DataLoaders**: 
  - **Training**: Batches of 32 images with shuffling enabled.
  - **Validation**: Batches of 32 images without shuffling.
  - **Parallelism**: Utilizes all available CPU cores for efficient data loading.

In [ ]:
from torch.utils.tensorboard import SummaryWriter
# Training Setup
model = InceptionRegressor().cuda()
criterion = nn.L1Loss()  # MAE Loss
optimizer = optim.Adam(model.parameters(), lr=1e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)
best_model_path = "best_model.pth"
num_epochs = 50
best_loss = float("inf")
early_stopping = EarlyStopping(patience=10, verbose=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
writer = SummaryWriter()

# Prepare Dataloaders
fold = 0  # Change fold index as needed
train_data = train_df[train_df["fold"] != fold].reset_index(drop=True)
valid_data = train_df[train_df["fold"] == fold].reset_index(drop=True)

dataset_train = SolarPanelDataset(train_data, transform=train_transforms)
dataset_valid = SolarPanelDataset(valid_data, transform=valid_transforms)

train_loader = DataLoader(dataset_train, batch_size=32, shuffle=True, num_workers=os.cpu_count())
valid_loader = DataLoader(dataset_valid, batch_size=32, shuffle=False)

In [17]:
import cv2
image = cv2.imread("/kaggle/input/lacuna-solar-survey-challenge/images/ID00rw8.jpg")
if image is not None:
    print("Image loaded successfully:", image.shape)
else:
    print("Failed to load image.")

Image loaded successfully: (3956, 5280, 3)


## 🚀 Model Training Loop

The model is trained over a maximum of 50 epochs with early stopping and dynamic learning rate adjustment to optimize performance.

### 🔄 Training and Validation Steps
- **Training Phase**:
  - Model set to training mode (`model.train()`).
  - For each batch:
    - Inputs and targets are loaded onto the device.
    - Forward pass is performed.
    - L1 loss is computed.
    - Gradients are backpropagated, and the optimizer updates the model parameters.
- **Validation Phase**:
  - Model set to evaluation mode (`model.eval()`).
  - No gradient computations (`torch.no_grad()`).
  - Loss is computed for validation data.

### 📈 Monitoring and Optimization
- **Loss Tracking**:
  - Both training and validation losses are averaged and logged each epoch.
- **Early Stopping**:
  - Stops training if no improvement is observed for 10 consecutive validation evaluations.
- **Learning Rate Scheduling**:
  - Reduces learning rate when the validation loss plateaus to fine-tune learning.
- **TensorBoard Logging**:
  - Loss curves for both training and validation phases are recorded using TensorBoard's `SummaryWriter`.

### 📝 Notes
- The best model is saved automatically during training whenever a new lowest validation loss is achieved.
- If early stopping criteria are met, the training process halts early to prevent overfitting.

In [ ]:
num_epochs = 50
best_loss = float("inf")
early_stopping = EarlyStopping(patience=10, verbose=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# TensorBoard writer
writer = SummaryWriter()


# Training Loop
for epoch in range(num_epochs):
    # Training Phase
    model.train()
    epoch_loss = 0.0
    for images, targets in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Training"):
        images, targets = images.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    # Validation Phase
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for images, targets in tqdm(valid_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Validation"):
            images, targets = images.to(device), targets.to(device)
            outputs = model(images)
            loss = criterion(outputs, targets)
            val_loss += loss.item()

    # Average Loss
    train_loss = epoch_loss / len(train_loader)
    val_loss /= len(valid_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

    # Save Best Model
    early_stopping(val_loss, model, best_model_path)
    if early_stopping.early_stop:
        print("Early stopping")
        break

    # Adjust learning rate
    scheduler.step(val_loss)

    # Log to TensorBoard
    writer.add_scalar('Loss/train', train_loss, epoch)
    writer.add_scalar('Loss/val', val_loss, epoch)

# Close the TensorBoard writer
writer.close()

Epoch 1/50 - Validation: 100%|██████████| 15/15 [02:10<00:00,  8.68s/it]


Epoch 1/50, Train Loss: 2.5297, Val Loss: 2.5917
Validation loss decreased (-2.591667 --> 2.591667).  Saving model ...


Epoch 2/50 - Validation: 100%|██████████| 15/15 [01:44<00:00,  6.95s/it]


Epoch 2/50, Train Loss: 2.4085, Val Loss: 2.5452
Validation loss decreased (-2.545186 --> 2.545186).  Saving model ...


Epoch 3/50 - Validation: 100%|██████████| 15/15 [01:39<00:00,  6.66s/it]


...

Epoch 17/50 - Validation: 100%|██████████| 15/15 [01:45<00:00,  7.06s/it]


Epoch 17/50, Train Loss: 1.7698, Val Loss: 2.2128


Epoch 18/50 - Validation: 100%|██████████| 15/15 [01:45<00:00,  7.03s/it]


Epoch 18/50, Train Loss: 1.7957, Val Loss: 2.2559


Epoch 19/50 - Training:  10%|█         | 9/89 [00:37<05:53,  4.42s/it]

## 📊 Model Evaluation

After training, the best-performing model (based on validation loss) is loaded and evaluated on the validation dataset.

### 🔍 Steps Performed
- **Model Loading**:
  - The model's parameters are restored from the saved checkpoint (`best_model.pth`).
- **Prediction**:
  - The model is set to evaluation mode.
  - Forward passes are performed on the validation set without updating gradients.
  - Predictions and true labels are collected for evaluation.
- **Evaluation Metric**:
  - The performance is assessed using **Mean Absolute Error (MAE)**, providing a straightforward interpretation of the model's average prediction error.

### 📈 Output
- Displays the final **Validation MAE**, indicating the average number of panels/boilers the model is off by.




In [ ]:
# Load Best Model
model.load_state_dict(torch.load(best_model_path))
model.eval()

# Predict on Validation Set
preds = []
true_vals = []
with torch.no_grad():
    for images, targets in tqdm(valid_loader, desc="Predicting on Validation Set"):
        images = images.cuda()
        outputs = model(images).cpu().numpy()
        preds.append(outputs)
        true_vals.append(targets.numpy())
preds = np.concatenate(preds, axis=0)
true_vals = np.concatenate(true_vals, axis=0)

from sklearn.metrics import mean_absolute_error

# Evaluate using MAE
mae = mean_absolute_error(true_vals, preds)
print(f"Validation MAE: {mae:.4f}")

## 🚀 Generating Test Predictions and Submission

After validating the model, predictions are generated for the unseen **test set**.

### 🛠️ Steps Performed
- **Prepare Test Data**:
  - Test images are read and preprocessed using the validation transforms.
- **Model Inference**:
  - The trained model predicts the number of boilers and panels for each test image.
- **Submission File Creation**:
  - Predictions are formatted according to the competition submission rules:
    - For each image ID, two rows are created: one for boilers (`_boil`) and one for panels (`_pan`).
  - Predicted values are clipped between **0** and **1000** to ensure reasonable outputs.
- **Saving Submission**:
  - A `Submission.csv` file is generated and saved for final submission.

### 📄 Output
- `Submission.csv` containing model predictions ready for evaluation.

In [ ]:
# Predict on Test Set
test_df = pd.read_csv("/kaggle/input/lacuna-solar-survey-challenge/Test.csv")

test_df["path"] = "/kaggle/input/lacuna-solar-survey-challenge/images/" + test_df["ID"] + ".jpg"

dataset_test = SolarPanelDataset(test_df, transform=valid_transforms, to_train=False)
test_loader = DataLoader(dataset_test, batch_size=32, shuffle=False)

test_preds = []
with torch.no_grad():
    for images in tqdm(test_loader, desc="Predicting on Test Set"):
        images = images.cuda()
        outputs = model(images).cpu().numpy()
        test_preds.append(outputs)
test_preds = np.concatenate(test_preds, axis=0)

# Create Sample Submission
submission = pd.DataFrame()
submission["ID"] = np.repeat(test_df["ID"].values, 2)
submission["ID"] = submission["ID"] + np.tile(["_boil", "_pan"], len(test_df))
submission["Target"] = test_preds.flatten().clip(0,1000)

# Save Submission
submission.to_csv("Submission.csv", index=False)
print("Submission saved!")